# Parameter Estimation Example

This tests pyomo's `paramest` module using the geothermal data of the preheater.
We have temperatures and pressures, and I'm going to assume the motive fluid is propane (I think it's actually like butane or pentane or something tho) and the brine is modelled as water.


In [ ]:
import pickle
import pandas
from pyomo.environ import ConcreteModel, SolverFactory, SolverStatus, TerminationCondition, Block, TransformationFactory
from pyomo.network import SequentialDecomposition, Port, Arc
from pyomo.core.base.units_container import _PyomoUnit, units as pyomo_units
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import report_statistics, degrees_of_freedom
import idaes.logger as idaeslog
from property_packages.build_package import build_package
from idaes.models.unit_models.pressure_changer import Compressor
from idaes.models.properties.general_helmholtz import HelmholtzParameterBlock, PhaseType, StateVars, AmountBasis
from pyomo.environ import Var, Objective, value, Constraint
from property_packages.build_package import build_package
from idaes.models.unit_models.heat_exchanger import HeatExchanger

# Build the model

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
 
# Set up property packages
m.fs.water = build_package("helmholtz",["water"],["Vap","Liq"])
m.fs.propane = build_package("helmholtz",["propane"],["Vap","Liq"]) 


m.fs.hx = HeatExchanger(
    hot_side_name="water",
    cold_side_name="propane",
    water={"property_package":m.fs.water},
    propane={"property_package":m.fs.propane},
)

# Setup the inlet conditions that will be constant
m.fs.hx.water_inlet.pressure.fix(1e6)  # Pa
m.fs.hx.propane_inlet.pressure.fix(5e5)  # Pa

# Heat Exchanger area and Heat Transfer Coefficient work together
# to determine the overall heat transfer. Having both as variables 
# would create redundancy and many solutions would be possible.
# We will fix the area and leave U as a variable.
m.fs.hx.area.fix(10)  # m^2

# The property package requires enth_mol as a state variable, we will create a variable
# for inlet temperature and use a constraint to link it to enth_mol
m.fs.hx.water_inlet_temp = Var(m.fs.time,initialize=400, units=pyomo_units.K)
@m.fs.hx.Constraint(m.fs.time)
def water_inlet_temp_rule(b, t):
    return b.hot_side.properties_in[t].temperature == b.water_inlet_temp[t]

m.fs.hx.propane_inlet_temp = Var(m.fs.time, initialize=300, units=pyomo_units.K)
@m.fs.hx.Constraint(m.fs.time)
def propane_inlet_temp_rule(b, t):
    return b.cold_side.properties_in[t].temperature == b.propane_inlet_temp[t]

# This model now has the following degrees of freedom:
# - Inlet Flow rates (water and propane)
# - Inlet Temperatures (water and propane)
# - Heat Transfer Coefficient U

# We will use parameter estimation from various inlet conditions and outlet conditions
# to estimate U.


In [5]:
# Load the data
with open('../reconcile_all.pk1', 'rb') as f:
    df = pickle.load(f)

# Inlet Conditions:
# Water inlet flow: BCOCF
# Water Inlet temp: we will just uses SIT even though that's not correct
# Water Outlet Temp: PHBOT
# Propane Inlet Temp: PHMFIT
# Propane Outlet Temp: PHMFOT
# We have to guess the propane inlet flow (either constant or proportional to water flow?)


/tmp/ipykernel_2776429/730780287.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f)


In [6]:
import pyomo.contrib.parmest.parmest as parmest
from pyomo.contrib.parmest.experiment import Experiment